## Imports, utils and setup

In [1]:
from IPython.display import display
import pandas as pd
import plotly.express as px
from transformers import pipeline

In [3]:
def sum_model_trainable_parameters(pytorch_model) -> int:
    return sum(p.numel() for p in pytorch_model.parameters())


def show_model_summary(model_name, model, model_size_mb):
    params_count = sum_model_trainable_parameters(model)
    return pd.Series(dict(
        parameters=f'{params_count:,}',
        size=f'{model_size_mb} MB'
    ), name=model_name).to_frame()

# Generating fluent text

## Preparation

Model site: https://huggingface.co/gpt2

In [6]:
from transformers import pipeline

text_generation_model_name = 'gpt2'
text_generation_pipeline = pipeline('text-generation',
                                    model=text_generation_model_name)
text_generation_model_size_mb = 523

In [7]:
show_model_summary(text_generation_model_name,
                   text_generation_pipeline.model, 
                   text_generation_model_size_mb)

,gpt2
parameters,"124,439,808"
size,523 MB


## Demo

In [8]:
from transformers import pipeline, set_seed
from pprint import pprint

set_seed(1)

text_generation_pipeline("Once upon the time",
                         max_length=25,
                         num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Once upon the time when he felt the greatest desire, then, by his infinite goodness of self, could feel him, then'},
 {'generated_text': "Once upon the time of the eclipse, a mass of energy from the sun's surface exploded throughout the sky and covered most of"}]

In [9]:
set_seed(2)

text_generation_pipeline("Once upon the time", 
                         max_length=25, 
                         num_return_sequences=2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Once upon the time when there is an earthquake, people usually go out into the street not to feel anxious as they would when'},
 {'generated_text': 'Once upon the time frame of 8.4.09, they are able to move to the next stage of the process.'}]

# Question answering

Model site: https://huggingface.co/deepset/roberta-base-squad2


## Preparation

In [10]:
qa_model_name = 'deepset/roberta-base-squad2'
qa_pipeline = pipeline('question-answering', qa_model_name)
qa_model_size_mb = 473

In [11]:
show_model_summary(qa_model_name, qa_pipeline.model, qa_model_size_mb)

,deepset/roberta-base-squad2
parameters,"124,056,578"
size,473 MB


In [12]:
def answer_question(question, context, qa_pipeline):
    return qa_pipeline(dict(
        question=question,
        context=context
    ))

In [13]:
australia_context = """Australia, officially the Commonwealth of Australia, 
    is a sovereign country comprising the mainland of the Australian continent, 
    the island of Tasmania, and numerous smaller islands.
    It is the largest country by area in Oceania and the world's sixth-largest country.
    Australia's population of nearly 26 million, in an area of 7,617,930 square 
    kilometres (2,941,300 sq mi),[14] is highly urbanised and heavily concentrated on 
    the eastern seaboard. Canberra is the nation's capital, while the largest city is 
    Sydney, and other major metropolitan areas include Melbourne, Brisbane, Perth, 
    and Adelaide."""

In [14]:
question_variants = [
    'How many citizens Australia has?',
    'How many people live in Australia?',
    'For how many people is Australia home?',
]

### Demo

In [16]:
for question in question_variants:
    result = answer_question(question=question,
                             context=australia_context,
                             qa_pipeline=qa_pipeline)
    answer = result['answer']
    confidence_score = round(result['score']*100)
    print(f'{question} {answer}. Confidence: {confidence_score}%')

How many citizens Australia has? 26 million. Confidence: 55%
How many people live in Australia? 26 million. Confidence: 52%
For how many people is Australia home? 26 million. Confidence: 53%


#### Answer is a string between 'start' and 'end' positions in a question

In [17]:
result

{'score': 0.534076452255249, 'start': 318, 'end': 328, 'answer': '26 million'}

In [18]:
australia_context[ result['start'] : result['end'] ]

'26 million'

# Summarization

## Generate summary

### Preparation

In [20]:
summarize_model_name="facebook/bart-large-cnn"
summarize_pipeline = pipeline("summarization", model=summarize_model_name)
summarize_model_size_mb=1510

In [21]:
show_model_summary(summarize_model_name, summarize_pipeline.model, summarize_model_size_mb)

,facebook/bart-large-cnn
parameters,"406,290,432"
size,1510 MB


### Demo

In [22]:
australia_context = """Australia, officially the Commonwealth of Australia, 
    is a sovereign country comprising the mainland of the Australian continent, 
    the island of a
    Tasmania, and numerous smaller islands.
    It is the largest country by area in Oceania and the world's sixth-largest country.
    Australia's population of nearly 26 million, in an area of 7,617,930 square 
    kilometres (2,941,300 sq mi),[14] is highly urbanised and heavily concentrated on 
    the eastern seaboard. Canberra is the nation's capital, while the largest city is 
    Sydney, and other major metropolitan areas include Melbourne, Brisbane, Perth, 
    and Adelaide."""

In [23]:
summarize_pipeline(australia_context, max_length=50, min_length=30, do_sample=False)

[{'summary_text': "Australia, officially the Commonwealth of Australia, is a sovereign country. It is the largest country by area in Oceania and the world's sixth-largest country. Australia's population of nearly 26 million, in an area of 7,"}]

## Generate title for article


### Preparation

In [25]:
headline_generator_model_name = "Michau/t5-base-en-generate-headline"
headline_generator_pipeline = pipeline('text2text-generation',
                                       model=headline_generator_model_name)
headline_generator_model_size_mb = 850

In [26]:
show_model_summary(headline_generator_model_name,
                   headline_generator_pipeline.model, 
                   headline_generator_model_size_mb)

,Michau/t5-base-en-generate-headline
parameters,"222,903,552"
size,850 MB


### Demo

In [27]:
article = """
To make observations in the infrared spectrum, JWST must be kept under 50 K 
(−223.2 °C; −369.7 °F); otherwise, infrared radiation from the telescope itself
would overwhelm its instruments. It therefore uses a large sunshield to block light 
and heat from the Sun, Earth, and Moon, and its position near the Sun-Earth L2 keeps 
all three bodies on the same side of the spacecraft at all times.[56] Its halo orbit 
around the L2 point avoids the shadow of the Earth and Moon, maintaining a constant 
environment for the sunshield and solar arrays.[54] The shielding maintains a stable 
temperature for the structures on the dark side, which is critical to maintaining precise
alignment of the primary mirror segments in space.[18]. The five-layer sunshield, 
each layer as thin as a human hair,[57] is constructed from Kapton E, a commercially 
available polyimide film from DuPont, with membranes specially coated with aluminum on 
both sides and a layer of doped silicon on the Sun-facing side of the two hottest layers 
to reflect the Sun's heat back into space.[18] Accidental tears of the delicate film 
structure during testing in 2018 were among the factors delaying the project.[58]
The sunshield was designed to be folded twelve times so that it fit within the Ariane
5 rocket's payload fairing, which is 4.57 m (15.0 ft) in diameter, and 16.19 m (53.1 ft)
long. The shield's fully deployed dimensions were planned as 14.162 m × 21.197 m 
(46.46 ft × 69.54 ft). The sunshield was hand-assembled at ManTech (NeXolve) in 
Huntsville, Alabama, before it was delivered to Northrop Grumman in Redondo Beach,
California, for testing.[59]. Because of the sunshield, JWST does not have an unlimited
field of regard at any given time. The telescope can see 40 percent of the sky from one 
position and can see all of the sky over a period of six months,[60] the amount of time 
it takes to complete half its orbit around the Sun."""

In [28]:
headline_generator_pipeline(article)

[{'generated_text': "JWST's Sunshield is a Spectroscopic Observ"}]

# Text classification

## Sentiment analysis
Model site: https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english

### Preparation

In [29]:
sentiment_model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
sentiment_pipeline = pipeline('sentiment-analysis', model=sentiment_model_name)
sentiment_model_size_mb = 255

In [30]:
show_model_summary(sentiment_model_name, sentiment_pipeline.model, sentiment_model_size_mb)

,distilbert-base-uncased-finetuned-sst-2-english
parameters,"66,955,010"
size,255 MB


### Demo

Watch how well sacrasm is detected

In [31]:
sentiment_pipeline('I love being served hot food')

[{'label': 'POSITIVE', 'score': 0.9989325404167175}]

In [32]:
sentiment_pipeline('I love being served cold food')

[{'label': 'NEGATIVE', 'score': 0.7160998582839966}]

## Emotion classification

### Preparation

In [33]:
emotion_model = 'bhadresh-savani/distilbert-base-uncased-emotion'
emotion_pipeline = pipeline('sentiment-analysis', model=emotion_model,
                            return_all_scores=True)
emotion_model_size_mb = 255

In [34]:
show_model_summary(emotion_model, emotion_pipeline.model,
                   emotion_model_size_mb)

,bhadresh-savani/distilbert-base-uncased-emotion
parameters,"66,958,086"
size,255 MB


### Demo

In [36]:
emotion_pipeline('Its something great being human.')

[[{'label': 'sadness', 'score': 0.000782029062975198},
  {'label': 'joy', 'score': 0.9968018531799316},
  {'label': 'love', 'score': 0.0003343094140291214},
  {'label': 'anger', 'score': 0.0009681519004516304},
  {'label': 'fear', 'score': 0.0004745248006656766},
  {'label': 'surprise', 'score': 0.0006392347277142107}]]

In [37]:
emotion_pipeline("I didn't expect that at all. It must be a joke...")

[[{'label': 'sadness', 'score': 0.020250950008630753},
  {'label': 'joy', 'score': 0.06607986986637115},
  {'label': 'love', 'score': 0.0011429135920479894},
  {'label': 'anger', 'score': 0.8615054488182068},
  {'label': 'fear', 'score': 0.0479285903275013},
  {'label': 'surprise', 'score': 0.0030922212172299623}]]

## Zero shot classification
Classify by any label

### Preparation

In [39]:
from transformers import pipeline

zero_shot_model_name = "facebook/bart-large-mnli"
zero_shot_classifier = pipeline("zero-shot-classification",
                                model=zero_shot_model_name)
zero_shot_model_size_mb = 1520

In [40]:
show_model_summary(zero_shot_model_name, zero_shot_classifier.model, zero_shot_model_size_mb)

,facebook/bart-large-mnli
parameters,"407,344,131"
size,1520 MB


### Demo

In [43]:
australia_description = """Australia, officially the Commonwealth of Australia, 
is a sovereign country comprising the mainland of the Australian continent, 
the island of Tasmania, and numerous smaller islands."""

gradient_description = """In vector calculus, the gradient of a scalar-valued differentiable function f of several
variables is the vector field (or vector-valued function) whose value at a point p is the 
vector[a] whose components are the partial derivatives"""

plankton_description = """Plankton are the diverse collection of organisms found in water 
(or air) that are unable to propel themselves against a current (or wind). The individual 
organisms constituting plankton are called plankters."""

brave_new_world_description = """Brave New World is a dystopian social science fiction novel by 
Aldous Huxley, written in 1931 and published in 1932. Largely set in a futuristic World State, whose citizens are
environmentally engineered into an intelligence-based social hierarchy."""

classes = ['geography', 'math',  'biology', 'literature']

texts_to_classify = [australia_description, gradient_description,
                     plankton_description, brave_new_world_description]

zero_shot_predictions = zero_shot_classifier(texts_to_classify,
                                             classes)

In [44]:
import pandas as pd

index = []
scores = []
for prediction in zero_shot_predictions:
    
    index.append(prediction['sequence'])
    scores_rounded = [round(s,3) for s in prediction['scores']]
    scores_dict = dict(zip(prediction['labels'], scores_rounded))
    scores.append(scores_dict)

pd.DataFrame(scores, index=index, columns=classes)

,geography,math,biology,literature
"Australia, officially the Commonwealth of Australia, \nis a sovereign country comprising the mainland of the Australian continent, \nthe island of Tasmania, and numerous smaller islands.",0.877,0.035,0.045,0.043
"In vector calculus, the gradient of a scalar-valued differentiable function f of several\nvariables is the vector field (or vector-valued function) whose value at a point p is the \nvector[a] whose components are the partial derivatives",0.011,0.967,0.007,0.015
Plankton are the diverse collection of organisms found in water \n(or air) that are unable to propel themselves against a current (or wind). The individual \norganisms constituting plankton are called plankters.,0.042,0.020,0.899,0.039
"Brave New World is a dystopian social science fiction novel by \nAldous Huxley, written in 1931 and published in 1932. Largely set in a futuristic World State, whose citizens are\nenvironmentally engineered into an intelligence-based social hierarchy.",0.010,0.003,0.004,0.983


## Zero shot vs sentiment specific model

In [45]:
classify_sentiment_texts = [
    'I love being served hot food',
    'I love being served cold food'
]

In [46]:
sentiment_model_predictions = sentiment_pipeline(classify_sentiment_texts, 
                                                 return_all_scores=True)
sentiment_model_predictions

[[{'label': 'NEGATIVE', 'score': 0.0010674481745809317},
  {'label': 'POSITIVE', 'score': 0.9989325404167175}],
 [{'label': 'NEGATIVE', 'score': 0.7160998582839966},
  {'label': 'POSITIVE', 'score': 0.28390011191368103}]]

In [47]:
labels = list(sentiment_pipeline.model.config.label2id.keys())
labels

['NEGATIVE', 'POSITIVE']

In [48]:
zeroshot_model_predictions = zero_shot_classifier(classify_sentiment_texts, labels)
zeroshot_model_predictions

[{'sequence': 'I love being served hot food',
  'labels': ['POSITIVE', 'NEGATIVE'],
  'scores': [0.8949331045150757, 0.10506691783666611]},
 {'sequence': 'I love being served cold food',
  'labels': ['POSITIVE', 'NEGATIVE'],
  'scores': [0.7917080521583557, 0.20829196274280548]}]

# Conversational

https://huggingface.co/microsoft/DialoGPT-small

## Preparation

In [51]:
conversational_model_model_name = 'microsoft/DialoGPT-large'
conversational_pipeline = pipeline('conversational', model=conversational_model_model_name)
conversational_model_size_mb = 1630

In [52]:
show_model_summary(conversational_model_model_name,
                   conversational_pipeline.model, 
                   conversational_model_size_mb)

,microsoft/DialoGPT-large
parameters,"774,030,080"
size,1630 MB


In [50]:
from transformers import Conversation

## Demo

In [53]:
conversation_1 = Conversation("Can I add fries?")
conversational_pipeline(conversation_1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Conversation id: 3ab1f1e5-be6b-4ddd-9b0b-a0821625c6ad 
user >> Can I add fries? 
bot >> Of course! 

In [54]:
conversation_1.add_user_input("How much is it?")
conversational_pipeline(conversation_1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Conversation id: 3ab1f1e5-be6b-4ddd-9b0b-a0821625c6ad 
user >> Can I add fries? 
bot >> Of course! 
user >> How much is it? 
bot >> I think it's like 3.50 

In [55]:
conversation_1.add_user_input("Ok, so I will order two")
conversational_pipeline(conversation_1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Conversation id: 3ab1f1e5-be6b-4ddd-9b0b-a0821625c6ad 
user >> Can I add fries? 
bot >> Of course! 
user >> How much is it? 
bot >> I think it's like 3.50 
user >> Ok, so I will order two 
bot >> I will order two as well. 